This script should start after the user has plotted the UAS ortho in QGIS, visually identified field plot centers (saved as shapefile), and cropped the largest rectangle possible from UAS ortho

Conda Environment: python 3.8, Windows 10

conda install -c conda-forge geopandas matplotlib mpld3 scipy rasterio 

In [32]:
import geopandas as gpd
#import pandas as pd
import numpy as np
from scipy import spatial
import rasterio as rio
import rasterio.mask
from shapely import geometry
from rasterio import MemoryFile

In [33]:
#file paths to UAS rectangle clip and field point centers

fp_uas = r"E:\USGS\Sonoma_fires\GLA3\GLA3_2021_ortho_clip_32610.tif"
fp_pts = r"E:\USGS\Sonoma_fires\GLA3\GLA3_2021_ctrs_32610.gpkg"

In [34]:
gdf = gpd.read_file(fp_pts)
#gdf.crs
mask = gdf['buffer'] = gdf.geometry.buffer(0.04) #buffer for masking plot markers

In [35]:
with rio.open(fp_uas) as src:
    #saving profile for writing out later
    profile = src.profile
    
    #masking the centers of field plots to remove yellow markers
    out_image, out_transform = rasterio.mask.mask(src, mask, invert=True, filled=False)

    #creating a new extent rounded to nearest 10m for aligning with Sent2 resolution
    #new_bb = [round(num,-1) for num in src.bounds]

#set masked pixels to np.nan  (for interpolation)
masked = np.where(out_image.mask, np.nan, out_image)

#the UAS ortho contains a 4th band we are not going to use
rgb = masked[:3]

In [36]:
#IDW interpolator
#should investigate faster interpolators for this small of an area

def idw(data):
    valid = np.argwhere(~np.isnan(data))
    tree = spatial.cKDTree(valid)
    
    nans = np.argwhere(np.isnan(data))    
    for row in nans:
        d, idx = tree.query(row, k=12) #k = number of nearest neighbors
        d = np.power(d, -2) #each item in d raised to its reciprocated power (basis of idw) the value "r" also defines the smoothness of the interpolation
        v = data[valid[idx, 0], valid[idx, 1]] 
        data[row[0], row[1]] = np.inner(v, d)/np.sum(d) #nans are replaced with the result of (v * d)/sum(d)
        
    return data

In [37]:
bands = []

#function to loop through bands and interpolate
def rgb_interp(data):
    for r in range(data.shape[0]):
        b = idw(data[r])
        bands.append(b.astype('uint8'))

    return bands

In [38]:
data_interp = rgb_interp(rgb)

arr = np.asarray(data_interp)

In [41]:
#arr.shape

(3, 12006, 14947)

In [39]:
#changing the count to 3 since we dropped the 4th band

profile.update(
        dtype=rasterio.uint8,
        count=3,
        compress='lzw')

In [43]:
outfile = fp_uas.replace('.tif','_interp.tif')

with rio.open(outfile, "w", **profile) as dest:
    dest.write(arr)

code below was for clipping UAS ---- which I prolly won't do until after creating new extent from Sentinel2 imagery first?

In [ ]:
#uas_clip_extent = geometry.Polygon([[new_bb[0]+10,new_bb[3]],
#                                    [new_bb[2]-10,new_bb[3]],
#                                    [new_bb[2]-10,new_bb[1]+10],
#                                    [new_bb[0]+10,new_bb[1]+10]])

#Rasterio requires a shapely polygon under the "geometry" header
#geo = gpd.GeoDataFrame({'geometry':uas_clip_extent},index=[0],crs="epsg:26910") 

In [ ]:
with MemoryFile() as memfile:
    with memfile.open(**profile) as dataset:
        dataset.write(arr)
 
        #print(dataset.profile)
        out_image2, out_transform2 = rasterio.mask.mask(dataset, geo.geometry, crop=True)
        out_meta = dataset.meta.copy()

    out_meta.update({
    "driver": "GTiff",
    "height": out_image2.shape[1],
    "width": out_image2.shape[2],
    "transform": out_transform2
    })

In [ ]:
outfile = fp_uas.replace('.tif','_2021_interp_clip_test3.tif')

with rio.open(outfile, "w", **out_meta) as dest:
    dest.write(out_image2)

In [ ]:
test = rio.open(r"E:\USGS\Sonoma_fires\GLA3\GLA3_26910_rect_2021_interp_clip_test3.tif")

In [ ]:
test.bounds

In [ ]:
test.close